# Continuous Portfolio Optimization

#### Device: Dirac-3


## Introduction

This approach seeks to identify a sub-portfolio of stocks that have superior risk-return profiles compared to the full portfolio. This identifies opportunities for an investor to simplify their investment strategy without sacrificing (and potentially enhancing) the risk-adjusted return. While the expected return on a portfolio is relatively straightforward to optimize by itself, optimizing against risk is more subtle when choosing a set of assets that individually have high returns. The reason that optimizing against risk is more challenging is because asset performances can be correlated. Intuitively, one can see that investing in two highly-correlated assets is more risky than if they are uncorrelated or even anti-correlated. If one of a highly correlated asset is performing poorly, the other in the pair is likely to do so as well. Thus, the variance of a portfolio with both assets can be significantly higher than it would be if they were uncorrelated. Minimizing the overall variance for the return of a portfolio therefore needs to take into account the covariance between the assets, making this a fundamentally quadratic problem, ideal for our Dirac-1 solver.

## Importance

Investments need not only give a good return on average, but also need to balance the potential risks. This balance will depend on the goal of the investor. For example, someone investing their retirement fund is likely to favor modest returns with low risks because the consequences of major losses are severe. On the other hand, someone who is day-trading in the hopes of having more money for entertainment might be willing to take more risks. There are of course many other scenarios with other risk levels. In general, the portfolio optimization problem is viewed as being a multi-objective problem. The goal is to balance the objective of maximizing return with the objective of minimizing risk. Even with multiple objectives there is still a sense of optimality, a portfolio is said to be "efficient" or "[Pareto optimal](https://en.wikipedia.org/wiki/Multi-objective_optimizationy)" if the only ways to decrease risk would be to also decrease return. Regardless of one's appetite for risk, it never makes sense to invest in a non-Pareto-Optimal portfolio, so our goal is to find those that are Pareto optimal and match the appetite for risk which is parameterized by a term  $\xi$ in our description. It is worth noting that this tutorial is based on a relatively simplified but still commonly used model of portfolio optimization. Various efforts exist to take into account more complex structure in the distribution of expected returns, in particular [the failure to capture extreme events](http://math.bu.edu/people/murad/pub/hist11-posted.pdf).

## Applications

Portfolios diversification is necessary to achieve satisfactory outcomes for investors, making the kind of portfolio optimization discussed here (and potentially more complex variants) highly important. In spite of its simplicity, the model of diversification presented here, often referred to as [modern portfolio](https://www.britannica.com/money/modern-portfolio-theory-explained) theory, is [still used](https://www.nutmeg.com/nutmegonomics/markowitzs-legacy-why-modern-portfolio-theory-still). Improvements of the models presented here comprise a subject known as [post-modern portfolio theory](http://actuaries.org/AFIR/Colloquia/Orlando/Ferguson_Rom.pdf). One improvement is to consider a quantity known as [downside risk](https://www.lehigh.edu/~xuy219/research/Downside.pdf) instead of variance. Downside risk only takes into account the risk of portfolio elements underperforming a goal, rather than their total variation. Since the goal of diversification is to protect from risk, this approach can yield better performance.

Continuous optimization helps in selecting and determining how much to invest in each stock. Unlike binary optimization, where you decide whether to include a stock or not (1 or 0), continuous optimization allows for more precise investment decisions by assigning continuous values to represent the proportion of your investment in each stock. This method helps create a well-diversified portfolio with varying levels of investment in different stocks to better manage risk and achieve desired returns.

![weighted stocks](figures/continuous/01.svg)

## Constructing our Objective Function

In our tutorial, we have historical datasets containing stock prices over a period of time. Our goal is to use these datasets to calculate important metrics such as [the rate of return](https://en.wikipedia.org/wiki/Rate_of_return), [expected return](https://en.wikipedia.org/wiki/Expected_return), [variance](https://en.wikipedia.org/wiki/Variance), and [covariance](https://en.wikipedia.org/wiki/Covariance). By calculating these metrics for each stock in our portfolio, we gain insights into their performance and risk characteristics. Expected return helps us gauge potential profitability, variance quantifies the risk associated with individual stocks, and covariance indicates how stocks move relative to each other. These calculations are essential for making informed investment decisions aimed at optimizing returns while managing risk effectively.


We define the following variables as:

**1. $i$:** Index of stock

**2. K:** Total number of available stocks in given portfolio.

**3. R:** Daily returns or the Returns per Day of the portfolio

**4. E(R):** Expected Return of the portfolio

**5. Var(R):** Variance of portfolio's returns

**6. $\xi$:** A parameter that balances the importance of maximizing returns versus minimizing risk.

**7. $R_b$:** Base interest rate.e; it is introduced so that the
two added terms in the objective function are on an equal footing.

**8. $w_i$:**  Weight of
stock i in the portfolio; $i \in {1, 2, ..., K}$


Now, we can define our **Objective Function**, i.e, the function that encapsulates the goal of constructing a portfolio that balances maximizing expected returns and minimizing risk.   as follows:


$$ \text{min}_{\{w_i\}_{i\in \{1,2,..k\}}}   -E(R) \cdot R_b + \xi\cdot \text{Var}(R) $$



We want our weights to sum to $100%$, therefore:

$$ \sum_{i=1}^k w_i = 100 $$



We also want the portfolio to be diverse, so we apply an upper limit on all the weights:

$$ w_i \leq W_{\text{max}} $$

In our example we set our  upper limit on all the weights $W_{\text{max}}$ equal to $80$.

## Hamiltonian Construction

We are going to get into more rigorous math in this section. So you can feel free to skip to the next section to understand the code flow and observe the results!


The portfolio daily return over a time period $m$ can be written as a linear combination of daily returns of its constituent stocks over the same time period. We have,

$$ R^{(m)}(t) = \sum_{i = 1}^K w_i r_i ^{(m)}(t) $$

where, $r_i ^{(m)}(t)$ is the daily return of stock $i$ at time $t$ during time period $m$.

The expectation of portfolio daily return over time period $m$ can thus be expanded as:
 $$ E(R^{(m)}) = \sum_{i = 1}^K w_i E(r_i ^{(m)}) $$

and the variance of portfolio daily returns over time period $m$ is expanded as,

$$ \text{Var}(R^{(m)}) = \sum_{i = 1}^K \sum_{j = 1}^K w_i w_j \text{Covar}(r_i ^{(m)},r_j ^{(m)}) $$

where Covar is the covariance function.



To impose the inequality constraints in $ w_i \leq W_{\text{max}}$, we can introduce $K$ auxiliary variables
$ w_{K+1}, w_{K+2}, ..., w_{2K} $, and impose $K$ equality constraints as,

$$ w_i + w_{i+K} = W_{\text{max}} $$


Substituting Equations for daily expectation of portfolio daily return over time period $m$ and variance of portfolio daily returns over time period $m$ into Equation 2, the objective function, that we now call $f^{(m)},$ becomes:

$$ f^{(m)}(w) = \sum_{i,j = 1} ^K w_i A_{ij}^{(m)}w_j + \sum_{i=1}^K w_ib_i^{(m)} + \alpha(\sum_{i=1}^K w_i - 100 )^2 + \beta(\sum_{i=1}^K (w_i + w_{i+K} - W_{\text{max}})^2 $$

where,

$$ A^{(m)}_{ij} = \xi\cdot \text{Covar}(r^{(m)}_i, r^{(m)}_j) $$

$$ b^{(m)}_i = −R_B\cdot E(r^{(m)}_i) $$


To avoid an over-fit on the portfolio data, we can minimize the average of the cost function over $M$
overlapping time periods, that is $m \in {1, 2, ..., M}.$ The problem becomes,

$$ f(w) =  \frac{1}{M} \sum_{m=1}^M f(w) $$


Adding the constraints to the objective function, the optimization
problem reduces to,

$$ \text{min}_{w_i} \sum_{i=1}^{2K} \sum_{j=1}^{2K} J_{ij} w_iw_j + \sum_{i=1}^{2k}h_iw_i $$

where the two body coupling coefficients $J_{ij}$ are defined as

$$ J_{ij}= J_{ij}^\mathrm{(obj)}+ \beta J_{ij}^\mathrm{(constr)} $$

where and objective function component of the coupling is defined as

$$ J_{ij}^\mathrm{(obj)} = 
  \begin{cases}
    \frac{1}{M} \sum_{m=1}^M A_{ij}^{(m)} + \alpha & \quad \text{if } i,j \leq K \\
    0 & \quad \text{otherwise}
  \end{cases} $$

and a constraint component defined as

$$ J_{ij}^\mathrm{(constr)} = 
  \begin{cases}
    1 & \quad \text{if } i=j \\
    1 & \quad \text{if } \left|i-j\right|=K \\
    0 & \quad \text{otherwise}
  \end{cases} $$


and the linear coefficients $h_i$ are defined as,

$$ h_{i}= h_{i}^\mathrm{(obj)}+ \beta h_{i}^\mathrm{(constr)} $$

where

$$ h_{i}^{(obj)} = 
  \begin{cases}
    b_i - 200\alpha & \quad \text{if } i \leq K \\
    0 & \text{otherwise}
  \end{cases} $$

and

$$ h_{i}^{(constr)} = - 2 W_{\text{max}} \quad \forall i $$

The Hamiltonian equation becomes:


$$ H = [ h \mid \frac{1}{2} (J + J^T)] $$

where:

- $( 0.5 \cdot (J + J^T) )$ represents the symmetrized form of $J$
- The notation $h \mid \frac{1}{2} (J + J^T)$ denotes the horizontal concatenation of $h$ and $\frac{1}{2} (J + J^T)$, in other words the addition of $h$ as a column of the matrix.

# Understanding the Code with an Example

Before we begin, you'll need your unique token to access the QCi Client API and connect to the Dirac device. If you don't have a token yet, you can sign up for our [Free Trial Cloud Access](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/quick-start-on-cloud)). Let's get started!

You can download the data sets and get access to the full code base [here](https://git.qci-dev.com/cvadlamani/basic-catalyst-tutorial/-/tree/Chitra/integer_portfolio_optimization?ref_type=heads)

Imagine you have  $5$  stocks in your portfolio and you want to find the optimized portfolio weights

Here are 5 stocks present in our portfolio, let us see how we can optimize them using our Run() method:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stock</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>AAPL</td>
    </tr>
    <tr>
      <th>1</th>
      <td>AMZN</td>
    </tr>
    <tr>
      <th>2</th>
      <td>GOOG</td>
    </tr>
    <tr>
      <th>3</th>
      <td>MSFT</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NVDA</td>
    </tr>
  </tbody>
</table>


## Run () method

The *run function* is the main function of the portfolio optimization pipeine that performs a series of operations to analyze stock data and optimize a stock portfolio based on historical returns.

It does three main things: first, it retrieves the stock data. Then it preprocesses the data and  constructs a Hamiltonian matrix to balance returns and risks. Finally, it optimizes the stock portfolio and the function ultimately saves the stock allocation data to a specified CSV file and displays the optimized portfolio

Here is a dump of functions from the gitlab, this may not be the best place for them but want to get all the code into the notebook with some initial efforts to clean it up

In [ ]:
import time
from qci_client import QciClient
import numpy as np
import pandas as pd

In [ ]:
api_url = "https://api.qci-prod.com"
api_token = "token goes here"
client = QciClient(api_token=api_token, url=api_url)

In [ ]:


###### parameters #######

DROP_STOCKS = []
ALPHA = 5.0
BETA = 1.0
XI = 1.0
WINDOW_DAYS = 30
WINDOW_OVERLAP_DAYS = 15
IN_SAMPLE_DAYS = 60
OUT_OF_SAMPLE_DAYS = 30
WEIGHT_UPPER_LIMIT = 0.08
R_BASE = 0.05 / 365

MIN_DATE = pd.to_datetime("2023-06-15") 
MAX_DATE = pd.to_datetime("2024-03-31")

ALLOC_OUT_FILE = (
    "data/unequal_stock_allocations_xi_%s_wd_%d_olap_%d_ins_%d_oos_%d_dirac3_s2_rel1.csv"
    % (
        str(XI),
        WINDOW_DAYS,
        WINDOW_OVERLAP_DAYS,
        IN_SAMPLE_DAYS,
        OUT_OF_SAMPLE_DAYS,
    )
)

######## end parameters ####


def convert_hamiltonian_to_poly(J, h):
    import numpy as np
    assert J.shape[0] == J.shape[1]

    N = J.shape[0]
    poly_coefs = []
    poly_indices = []
    for i in range(N):
        for j in range(i, N):
            if i == j:
                coef = J[i][j]
            elif i < j:
                coef = J[i][j] + J[j][i]
            else:
                assert False, "Incorrect indices!"

            if coef == 0.0:
                continue

            poly_coefs.append(coef)
            poly_indices.append([i + 1, j + 1])

    for i in range(N):
        coef = h[i]

        if coef == 0.0:
            continue

        poly_coefs.append(coef)
        poly_indices.append([0, i + 1])

    assert len(poly_coefs) == len(poly_indices)

    poly_coefs = np.array(poly_coefs, dtype=np.float32)
    poly_indices = np.array(poly_indices, dtype=np.int32)

    return poly_coefs, poly_indices


def get_constituents(adj_date):
    import pandas as pd
    from parameters import DROP_STOCKS
    from remove_unavailable_stocks import remove_unavailable_stocks
    adj_date = pd.to_datetime(adj_date)
    file_path = "data/stock_data.csv"
    df = pd.read_csv(file_path)
    #df = pd.read_csv("data/stock_data.csv", low_memory=False)

    df["beg_date"] = df["beg_date"].apply(pd.to_datetime)

    beg_dates = sorted(df["beg_date"].unique(), reverse=True)

    for beg_date in beg_dates:
        if adj_date >= beg_date:
            break

    stocks = list(df[df["beg_date"] == beg_date]["symbol"].unique())
    stocks = list(set(stocks) - set(DROP_STOCKS))
    stocks = remove_unavailable_stocks(stocks, adj_date)

    return stocks
    

def get_hamiltonian(
    return_df,
    stocks,
    min_date,
    max_date,
    xi=XI,
    window_days=WINDOW_DAYS,
    window_overlap_days=WINDOW_OVERLAP_DAYS,
):
    import numpy as np
    import datetime
    import pandas as pd
    from parameters import R_BASE, WEIGHT_UPPER_LIMIT, ALPHA, BETA
    K = len(stocks)

    # Calculate P and Q
    Q = np.zeros(shape=(K, K), dtype=np.float32)
    #P = np.zeros(shape=(K, K), dtype=np.float32)
    p_vec = np.zeros(shape=(K), dtype=np.float32)
    
    m = 0
    min_date = pd.to_datetime(min_date)
    max_date = pd.to_datetime(max_date)
    tmp_date = min_date
    while tmp_date <= max_date:
        tmp_min_date = tmp_date
        tmp_max_date = tmp_date + datetime.timedelta(days=window_days)
        tmp_df = return_df[
            (return_df["Date"] >= tmp_min_date)
            & (return_df["Date"] <= tmp_max_date)
        ]

        r_list = []
        for i in range(K):
            r_list.append(np.array(tmp_df[stocks[i]]))

        Q_tmp = np.cov(r_list)
        for i in range(K):
            p_vec[i] += -R_BASE * np.mean(r_list[i])             
            for j in range(K):
                #P[i][j] += np.mean(r_list[i]) * np.mean(r_list[j])

                Q[i][j] += Q_tmp[i][j]

        tmp_date += datetime.timedelta(
            days=window_days - window_overlap_days,
        )
        m += 1

    fct = m
    if fct > 0:
        fct = 1.0 / fct

    #P = fct * P
    p_vec = fct * p_vec
    Q = fct * Q

    # Calculate the Hamiltonian
    #J_no_limit = -P + xi * Q
    J_no_limit = xi * Q

    # make sure J is symmetric up to machine precision
    J_no_limit = 0.5 * (J_no_limit + J_no_limit.transpose())

    #h_no_limit = np.zeros(shape=(K), dtype=np.float32)
    h_no_limit = p_vec
    
    if WEIGHT_UPPER_LIMIT is None:
        return J_no_limit, h_no_limit, 100.0

    W_max = 100.0 * WEIGHT_UPPER_LIMIT

    J = np.zeros(shape=(2 * K, 2 * K), dtype=np.float32)
    h = np.zeros(shape=(2 * K), dtype=np.float32)

    for i in range(K):
        for j in range(K):
            J[i][j] = J_no_limit[i][j] + ALPHA

        J[i][i] += BETA
        J[i][i + K] += BETA
        J[i + K][i] += BETA
        J[i + K][i + K] += BETA

        h[i] = h_no_limit[i] - 200.0 * ALPHA - 2 * BETA * W_max
        h[i + K] = -2 * BETA * W_max

    return J, h, K * W_max


def get_stock_returns(stocks, min_date, max_date):
    import pandas as pd
    min_date = pd.to_datetime(min_date)
    max_date = pd.to_datetime(max_date)
    return_df = None
    for stock in stocks:
    
        stock_df = pd.read_csv("data/stock_prices/%s.csv" % stock)
        stock_df["Date"] = stock_df["Date"].astype("datetime64[ns]")
        stock_df = (
            stock_df.fillna(method="ffill")
            .fillna(method="bfill")
            .fillna(0)
        )
        stock_df[stock] = stock_df[stock].pct_change()
        stock_df = stock_df.dropna()

        stock_df = stock_df[
            (stock_df["Date"] >= min_date) & (stock_df["Date"] <= max_date)
        ]

        if return_df is None:
            return_df = stock_df
        else:
            return_df = return_df.merge(
                stock_df,
                how="outer",
                on="Date",
            )

    return_df = (
        return_df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    )

    return return_df

def optimize_portfolio(J, h, sum_constraint, stocks, client):

    

    K = len(stocks)

    assert J.shape[0] == K or J.shape[0] == 2 * K
    assert J.shape[1] == K or J.shape[0] == 2 * K
    assert h.shape[0] == K or h.shape[0] == 2 * K


    h = h.reshape((h.shape[0], 1))
    H = np.hstack((h, 0.5 * (J + J.T)))

    ham_file = {
        "file_name": "port_opt_nasdaq100_d3",
        "file_config": {"hamiltonian": {"data": H}},
    }

    resp_json = client.upload_file(file=ham_file)
    file_id = resp_json["file_id"]
    job_tags = ["port_opot_nasdaq100_d3"]
    job_body = client.build_job_body(
        job_type="sample-hamiltonian",
        hamiltonian_file_id=file_id,
        job_params={
            "sampler_type": "dirac-3",
            "nsamples": 1,
            "solution_type": "continuous",
            "sum_constraint": sum_constraint,
            "relaxation_schedule": 1,
        },
        job_tags=job_tags,
    )
    response = client.process_job(
        job_body=job_body,
        wait=True,
    )
    print(response)

    sol = response["results"]["solutions"][0]

    assert (
        len(sol) == K or len(sol) == 2 * K
    ), "Inconsistent solution vector size!"

    weight_hash = {}
    for i in range(K):
        weight_hash[stocks[i]] = sol[i] / 100.0

    tot_weight = sum(weight_hash.values())

    print("Sum of weights:", tot_weight)

    if tot_weight != 1.0:
        print("Adjusting the weights...")

        for stock in stocks:
            weight_hash[stock] = weight_hash[stock] / tot_weight

    print("Adjusted weights:", weight_hash)


    return sol, weight_hash





def remove_unavailable_stocks(stocks, adj_date):
    import pandas as pd
    import datetime
    from parameters import IN_SAMPLE_DAYS, OUT_OF_SAMPLE_DAYS
    sel_stocks = []
    for stock in stocks:
        stock_df = pd.read_csv("data/stock_prices/%s.csv" % stock)

        if stock_df.shape[0] == 0:
            continue
        
        stock_df["Date"] = stock_df["Date"].astype("datetime64[ns]")

        adj_date = pd.to_datetime(adj_date)
        beg_date = adj_date - datetime.timedelta(days=IN_SAMPLE_DAYS)
        end_date = adj_date + datetime.timedelta(days=OUT_OF_SAMPLE_DAYS)

        stock_df = stock_df[
            (stock_df["Date"] >= beg_date) &
            (stock_df["Date"] <= end_date)
        ]

        if stock_df.shape[0] == 0:
            continue
        
        sel_stocks.append(stock)

    print("Chose %d of %d stocks" % (len(sel_stocks), len(stocks)))

    return sel_stocks


def remove_unavailable_stocks(stocks, adj_date):
    import pandas as pd
    import datetime
    from parameters import IN_SAMPLE_DAYS, OUT_OF_SAMPLE_DAYS
    sel_stocks = []
    for stock in stocks:
        stock_df = pd.read_csv("stock_prices/%s.csv" % stock)

        if stock_df.shape[0] == 0:
            continue
        
        stock_df["Date"] = stock_df["Date"].astype("datetime64[ns]")

        adj_date = pd.to_datetime(adj_date)
        beg_date = adj_date - datetime.timedelta(days=IN_SAMPLE_DAYS)
        end_date = adj_date + datetime.timedelta(days=OUT_OF_SAMPLE_DAYS)

        stock_df = stock_df[
            (stock_df["Date"] >= beg_date) &
            (stock_df["Date"] <= end_date)
        ]

        if stock_df.shape[0] == 0:
            continue
        
        sel_stocks.append(stock)

    print("Chose %d of %d stocks" % (len(sel_stocks), len(stocks)))

    return sel_stocks





![flow diagram](figures/continuous/02.svg)

## Step 1

Initialize and Prepare Dates:
* Calculates the in-sample date range based on the current date for historical data analysis.

In [ ]:
current_date  = pd.to_datetime(current_date )
in_sample_start_date  = current_date  - datetime.timedelta(days=in_sample_days)
in_sample_end_date  = current_date - datetime.timedelta(days=1)

Retrieve Stock Data:
* Obtains the list of stock constituents as of the current_date.
* Fetches historical stock returns for the in-sample period.

In [ ]:
stocks = get_constituents(current_date)
in_sample_returns_df  = get_stock_returns(stocks, in_sample_start_date, in_sample_end_date)

## Step 2
Preprocess Data:
* Sorts the stock return data by date.
* Fills any missing values in the stock return data.

In [ ]:
in_sample_returns_df  = in_sample_returns_df.sort_values("Date")
in_sample_returns_df  = in_sample_returns_df.fillna(method="ffill").fillna(0)

## Step 3
Calculates Elements of Hamiltonian
* Computes the elements $J$ and $h$, and a sum constraint using the in-sample stock returns.


In [ ]:
J, h, sum_constraint = get_hamiltonian(
    in_sample_returns_df,
    stocks,
    in_sample_start_date ,
    in_sample_end_date ,
)

#np.save("J_%s.npy" % current_date.strftime("%Y-%m-%d"), J)
#np.save("h_%s.npy" % current_date.strftime("%Y-%m-%d"), h)



## Step 4
* Optimize Portfolio:
  * Creates Hamiltonian using Hamiltonian data.
  * Runs a portfolio optimization algorithm using the Hamiltonian.
  * Saves the optimization results and stock list to files.


In [ ]:
  optimized_weights , stock_allocations = optimize_portfolio(J, h, sum_constraint, stocks)

The result JSON file looks like this:

In [3]:
{
  "job_info": {
    "job_id": "6679944a450099160ba0bf88",
    "job_submission": {
      "job_tags": ["port_opot_nasdaq100_d3"],
      "problem_config": {
        "normalized_qudit_hamiltonian_optimization": {
          "hamiltonian_file_id": "6679944a98263204a365f329"
        }
      },
      "device_config": {
        "dirac-3": {
          "num_samples": 1,
          "relaxation_schedule": 1,
          "sum_constraint": 40
        }
      }
    },
    "job_status": {
      "submitted_at_rfc3339nano": "2024-06-24T15:44:10.523Z",
      "queued_at_rfc3339nano": "2024-06-24T15:44:10.524Z",
      "running_at_rfc3339nano": "2024-06-24T15:44:11.514Z",
      "completed_at_rfc3339nano": "2024-06-24T15:44:13.339Z"
    },
    "job_result": {
      "file_id": "6679944d98263204a365f32b",
      "device_usage_s": 2
    }
  },
  "status": "COMPLETED",
  "results": {
    "counts": [1],
    "energies": [-32319.5527344],
    "solutions": [
      [8.0512924, 7.9148078, 7.9524417, 7.9162955, 8.1651621, 0, 0, 0, 0, 0]
    ]
  }
}

{'job_info': {'job_id': '6679944a450099160ba0bf88',
  'job_submission': {'job_tags': ['port_opot_nasdaq100_d3'],
   'problem_config': {'normalized_qudit_hamiltonian_optimization': {'hamiltonian_file_id': '6679944a98263204a365f329'}},
   'device_config': {'dirac-3': {'num_samples': 1,
     'relaxation_schedule': 1,
     'sum_constraint': 40}}},
  'job_status': {'submitted_at_rfc3339nano': '2024-06-24T15:44:10.523Z',
   'queued_at_rfc3339nano': '2024-06-24T15:44:10.524Z',
   'running_at_rfc3339nano': '2024-06-24T15:44:11.514Z',
   'completed_at_rfc3339nano': '2024-06-24T15:44:13.339Z'},
  'job_result': {'file_id': '6679944d98263204a365f32b', 'device_usage_s': 2}},
 'status': 'COMPLETED',
 'results': {'counts': [1],
  'energies': [-32319.5527344],
  'solutions': [[8.0512924,
    7.9148078,
    7.9524417,
    7.9162955,
    8.1651621,
    0,
    0,
    0,
    0,
    0]]}}

Where the following fields are:

- **job_info**: Contains details about the job.
  - **job_id**: Unique identifier for the job (`6679944a450099160ba0bf88`).
  - **job_submission**: Information about the job submission.
    - **job_tags**: Tags associated with the job (`port_opot_nasdaq100_d3`).
    - **problem_config**: Configuration of the problem to be solved.
      - **normalized_qudit_hamiltonian_optimization**: Specifies the problem type as normalized qudit Hamiltonian optimization.
        - **hamiltonian_file_id**: ID of the file containing the Hamiltonian information (`6679944a98263204a365f329`).
    - **device_config**: Configuration of the device used for the job.
      - **dirac-3**: Specifies the device used (`dirac-3`).
        - **num_samples**: Number of samples taken (value: 1).
        - **relaxation_schedule**: Relaxation schedule parameter (value: 1).
        - **sum_constraint**: Sum constraint parameter (value: 40).
  - **job_status**: Status updates of the job.
    - **submitted_at_rfc3339nano**: Time when the job was submitted (`2024-06-24T15:44:10.523Z`).
    - **queued_at_rfc3339nano**: Time when the job was queued (`2024-06-24T15:44:10.524Z`).
    - **running_at_rfc3339nano**: Time when the job started running (`2024-06-24T15:44:11.514Z`).
    - **completed_at_rfc3339nano**: Time when the job completed (`2024-06-24T15:44:13.339Z`).
  - **job_result**: Results of the job.
    - **file_id**: ID of the file containing the job result (`6679944d98263204a365f32b`).
    - **device_usage_s**: Time the device was used, in seconds (value: 2).

- **status**: The current status of the job (`COMPLETED`).

- **results**: The outcomes of the job.
  - **counts**: Number of samples taken for each solution (value: [1]).
  - **energies**: Energy values associated with the solutions (value: [-32319.5527344]).
  - **solutions**: The actual solutions found, represented as a list of values (value: [[8.0512924, 7.9148078, 7.9524417, 7.9162955, 8.1651621, 0, 0, 0, 0, 0]]).




The solution weights are then adjusted to form a balanced portfolio and we obtain:

```python
 Optimal_Portfolio_with_allocations = {
    "AAPL": 0.2012823125160289,
    "NVDA": 0.19787019747337747,
    "AMZN": 0.19881104498513807,
    "MSFT": 0.19790738997384238,
    "GOOG": 0.20412905505161316
  }

Where The optimal allocations for the portfolio are as follows:
  - **AAPL**: 20.13%
  - **NVDA**: 19.79%
  - **AMZN**: 19.88%
  - **MSFT**: 19.79%
  - **GOOG**: 20.41%

## Step 5
Generate and Save Allocation Data:
* Creates a DataFrame with stock allocations.
* Saves the allocation data to a specified CSV file, appending if the file already exists.

In [ ]:
np.save("qci_sol_%s.npy" % current_date.strftime("%Y-%m-%d"), sol)
np.save("stock_list_%s.npy" % current_date.strftime("%Y-%m-%d"), stocks)    
weight_df = pd.DataFrame(
    {
        "Stock": [item for item in stock_allocations.keys()],
        "Allocation": [
            stock_allocations[item] for item in stock_allocations.keys()
        ],
    }
)
weight_df["Date"] = current_date
weight_df = weight_df[weight_df["Allocation"] > 0]
weight_df["Stock_Count"] = weight_df.shape[0]
if os.path.exists(ALLOC_OUT_FILE):
    weight_df.to_csv(
        ALLOC_OUT_FILE,
        index=False,
        mode="a",
        header=False,
    )
else:
    weight_df.to_csv(
        ALLOC_OUT_FILE,
        index=False,
    )

Display optimized portfolio

In [ ]:
display(HTML(weight_df[["Stock","Allocation"]].tail(5).to_html()))


## Optimial Portfilio with Allocated weights:

When we run our code, we will see that the optimal portfolio has the following stock allocations:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stock</th>
      <th>Allocation</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NVDA</td>
      <td>0.196433</td>
    </tr>
    <tr>
      <th>1</th>
      <td>AMZN</td>
      <td>0.194561</td>
    </tr>
    <tr>
      <th>2</th>
      <td>MSFT</td>
      <td>0.211609</td>
    </tr>
    <tr>
      <th>3</th>
      <td>AAPL</td>
      <td>0.195888</td>
    </tr>
    <tr>
      <th>4</th>
      <td>GOOG</td>
      <td>0.201509</td>
    </tr>
  </tbody>
</table>

![weighted chart](figures/continuous/03.svg)

## Conclusion

In conclusion, we observe that Dirac effectively finds the solution and successfully solves the portfolio optimization problem. For a more challenging and comprehensive use case, refer to our example tutorial on optimizing a 100-asset portfolio over a 21-year period using the Nasdaq-100 as a benchmark with the Dirac-3 machine. Click [here](https://git.qci-dev.com/qci-dev/quantum-solutions/-/blob/main/qc/portfolio_optimization_dirac_3/build_unequal_nasdaq100_dirac3.py?ref_type=heads) to learn more.
